# Introdruction to DraCor Linked Open Data

*Ingo Börner* (ingo.boerner@uni-potsdam.de)

## DraCor Ontology

The onlology can be visualized using WebVowl: https://vowl.acdh.oeaw.ac.at/#iri=https://raw.githubusercontent.com/dracor-org/dracor-schema/ontology/ontology/dracor-ontology.xml

Run the following cell to display it here:

In [12]:
#Display the Ontology in an iframe
from IPython.display import IFrame
IFrame('https://vowl.acdh.oeaw.ac.at/#iri=https://raw.githubusercontent.com/dracor-org/dracor-schema/ontology/ontology/dracor-ontology.xml', width=1000, height=600)


## List available corpora

Run on DraCor staging: https://bit.ly/3sQd53y

In [1]:
"""
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX urn: <http://fliqz.com/>
PREFIX dracon: <http://dracor.org/ontology#>
SELECT ?corpus (COUNT(?play) AS ?cnt) FROM <https://dracor.clscor.io> WHERE {
  ?play dracon:in_corpus ?corpus .
}
GROUP BY ?corpus
ORDER BY DESC(?cnt)
"""

'\nPREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>\nPREFIX urn: <http://fliqz.com/>\nPREFIX dracon: <http://dracor.org/ontology#>\nSELECT ?corpus (COUNT(?play) AS ?cnt) FROM <https://dracor.clscor.io> WHERE {\n  ?play dracon:in_corpus ?corpus .\n}\nGROUP BY ?corpus\nORDER BY DESC(?cnt)\n'

## List connections to Wikidata
Run on DraCor staging: https://bit.ly/3CobyF2

In [2]:
"""
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX frbroo: <http://iflastandards.info/ns/fr/frbr/frbroo/>
PREFIX crm: <http://www.cidoc-crm.org/cidoc-crm/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX urn: <http://fliqz.com/>
PREFIX dracon: <http://dracor.org/ontology#>
SELECT ?play ?wd FROM <https://dracor.clscor.io> WHERE {
  ?play dracon:in_corpus <https://dracor.org/entity/corpus/ger> ;
        crm:P165_incorporates ?expression .
  ?expression frbroo:R40i_is_representative_expression_for ?work .
  
  ?work owl:sameAs ?wd .
}
LIMIT 500
"""

'\nPREFIX owl: <http://www.w3.org/2002/07/owl#>\nPREFIX frbroo: <http://iflastandards.info/ns/fr/frbr/frbroo/>\nPREFIX crm: <http://www.cidoc-crm.org/cidoc-crm/>\nPREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>\nPREFIX urn: <http://fliqz.com/>\nPREFIX dracon: <http://dracor.org/ontology#>\nSELECT ?play ?wd FROM <https://dracor.clscor.io> WHERE {\n  ?play dracon:in_corpus <https://dracor.org/entity/corpus/ger> ;\n        crm:P165_incorporates ?expression .\n  ?expression frbroo:R40i_is_representative_expression_for ?work .\n  \n  ?work owl:sameAs ?wd .\n}\nLIMIT 500\n'

## Locations of first performances (Wikidata)

In [ ]:
"""
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX frbroo: <http://iflastandards.info/ns/fr/frbr/frbroo/>
PREFIX crm: <http://www.cidoc-crm.org/cidoc-crm/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX urn: <http://fliqz.com/>
PREFIX dracon: <http://dracor.org/ontology#>
PREFIX wd: <http://www.wikidata.org/entity/>
PREFIX wdt: <http://www.wikidata.org/prop/direct/>
SELECT ?play ?playLabel ?wd ?ortLabel ?coords FROM <https://dracor.clscor.io> WHERE {
  ?play a dracon:play ;
    	crm:P165_incorporates ?expression ;
        rdfs:label ?playLabel ;
        dracon:in_corpus <https://dracor.org/entity/corpus/ger> .

  ?expression a frbroo:F22_Self-Contained_Expression ;
              frbroo:R40i_is_representative_expression_for ?work .

  ?work owl:sameAs ?wd .

  SERVICE <https://query.wikidata.org/sparql> {
  ?wd wdt:P4647 ?ort ;
       rdfs:label ?wdLabel .

    ?ort wdt:P625 ?coords ;
         rdfs:label ?ortLabel .

    FILTER (lang(?ortLabel) = "en")
    FILTER (lang(?wdLabel) = "en")
  }

  FILTER(lang(?playLabel) = "ger")
}
LIMIT 100

"""

## Characters across corpora

In [ ]:
"""
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX frbroo: <http://iflastandards.info/ns/fr/frbr/frbroo/>
PREFIX crm: <http://www.cidoc-crm.org/cidoc-crm/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX urn: <http://fliqz.com/>
PREFIX dracon: <http://dracor.org/ontology#>
PREFIX wd: <http://www.wikidata.org/entity/>
PREFIX wdt: <http://www.wikidata.org/prop/direct/>
SELECT ?characterWD (SAMPLE(?characterLabel) AS ?characterLabelSample) (COUNT(?character) AS ?cntCharacters) (COUNT(DISTINCT ?corpus) as ?corpusCnt) FROM <https://dracor.clscor.io> WHERE {
  ?character a dracon:character ;
             rdfs:label ?characterLabel ;
             frbroo:R57_is_based_on ?characterWD ;
             dracon:is_character_in ?play .

  ?play dracon:in_corpus ?corpus .

}
GROUP BY ?characterWD
ORDER BY DESC(?cntCharacters)
"""